In [4]:
import numpy as np
import math
import scipy as sp
from scipy import optimize
from numpy import genfromtxt
import pandas as pd 
import glob
import pymc as pm
import arviz as az
import statistics as stats
import matplotlib.pyplot as plt
import os,sys

In [13]:
# getting the name of the directory
# where the this file is present.
# current = os.path.dirname(os.path.realpath(__file__))
current = os.path.dirname(os.getcwd())
 
# Getting the parent directory name
# where the current directory is present.
# parent = os.path.dirname(os.path.dirname(current))
parent = current
print(parent)
#/Users/pizarror/IDM

# adding the parent directory to
# the sys.path.
sys.path.append(parent)

from IDM_model.src import model_functions as mf


/Users/pizarror/IDM


In [ ]:
def read_load_data(subject='23_IDM_0144',fn='/tmp',alpha0=1.0,cols=[]):
    df = pd.read_csv(fn)
    # remove practice trials
    df = df.loc[df['cdd_trial_type']=='task']
    # insert probability as choice into data
    # cols = ['cdd_trial_resp.corr','cdd_immed_amt','cdd_delay_amt','cdd_immed_wait','cdd_delay_wait','alpha']
    # also returns percent_reward which we do not need here
    data = mf.get_data(df,cols,alpha_hat=alpha0)[0]
    data['subject'] = subject
    # vn = data['cdd_immed_amt'].to_list()
    # vr = data['cdd_delay_amt'].to_list()
    # tn = data['cdd_immed_wait'].to_list()
    # tr = data['cdd_delay_wait'].to_list()
    # choice = data['cdd_trial_resp.corr'].to_list()
    return data # vn,vr,tn,tr,choice

## Complete Pooling

Complete pooling ignores the group-level information and considers all data as belonging to the same category. All groups are described with the same model. 

We are using complete pooling to generate priors for when we implement a higherarchical bayesian model.

### Load data

We will load the data from all participants so we can run the modeling schema

In [ ]:

# Takes about 16 seconds

# we will change this when we change utility to 1st level analysis (or split)
split_dir = '/Volumes/UCDN/datasets/IDM/split/'
save_dir = '/Volumes/UCDN/datasets/IDM/BH/csv'
subjs = sorted(glob.glob(os.path.join(split_dir,'23_IDM_*')))
task = 'cdd'
# VNa,VRa,TNa,TRa,Cha = [[]]*5
cols = ['cdd_trial_resp.corr','cdd_immed_amt','cdd_delay_amt','cdd_immed_wait','cdd_delay_wait','alpha']
data = pd.DataFrame(columns=['subject']+cols)

for s in subjs:
    subject = os.path.basename(s)
    fn  = os.path.join(s,task,'{}_{}.csv'.format(os.path.basename(s),task))
    if os.path.exists(fn):
        # SSA (Ben's smaller sooner amount) is vn (value_null)
        # LLA (Ben's larger later amount) is vr (value_reward)
        # SSD (Ben's smaller sooner delay) is tn (time_null)
        # LLD (Ben's larger later delay) is tr (time_reward)
        # y_pool (Ben's) is choice 
        # vn,vr,tn,tr,choice = read_load_data(fn=fn,alpha0=1.0)
        subj_data = read_load_data(subject=subject,fn=fn,alpha0=1.0,cols=cols)
        data = pd.concat([data,subj_data],ignore_index=True)
        # VNa = VNa + vn
        # VRa = VRa + vr
        # TNa = TNa + tn
        # TRa = TRa + tr
        # Cha = Cha + choice

print(data)





### 1. Simple Hierarchical Model

Pool all data together and run simple BH model


In [ ]:
subjects = data['subject'].unique()
subjects = subjects[:100]
print(subjects)
data = data.loc[data['subject'].isin(subjects)]

print(data)


In [ ]:

# takes approximately 6-10 ,minutes for 10 subjects

subjects = data['subject'].unique()
nb_subj = subjects.shape[0]
nb_trials = data.shape[0]//nb_subj
subj_id = np.array([ [s]*nb_trials for s in range(nb_subj) ]).flatten()

delay_amt = data['cdd_delay_amt'].values
delay_wait = data['cdd_delay_wait'].values
immed_amt = data['cdd_immed_amt'].values
immed_wait = data['cdd_immed_wait'].values
choices = data['cdd_trial_resp.corr'].values


# for iSubj,s in enumerate(subjects):
#     print('Working on modeling for subject : {}'.format(s))
#     delay_amt[iSubj,:] = data.loc[data['subject']==s]['cdd_delay_amt'].values
#     # if len(delay_amt)!=96:
#     #     print('Whoops dataset is not right size : check subject {}'.format(s))
#     #     sys.exit()
#     delay_wait[iSubj,:] = data.loc[data['subject']==s]['cdd_delay_wait'].values
#     immed_amt[iSubj,:] = data.loc[data['subject']==s]['cdd_immed_amt'].values
#     immed_wait[iSubj,:] = data.loc[data['subject']==s]['cdd_immed_wait'].values
#     choices[iSubj,:] = data.loc[data['subject']==s]['cdd_trial_resp.corr'].values
    
# print(delay_wait.shape)

In [ ]:
fn = os.path.join(save_dir,'completely_pooled_model.csv')
pool_model = pd.read_csv(fn,index_col=0)

mu_kappa_hat,std_kappa_hat = pool_model.loc['kappa[0]','mean'],pool_model.loc['kappa[0]','sd']
mu_gamma_hat,std_gamma_hat = pool_model.loc['gamma[0]','mean'],pool_model.loc['gamma[0]','sd']

print('For kappa, use the following (mu,sigma) : ({},{})'.format(mu_kappa_hat, std_kappa_hat))
print('For gamma, use the following (mu,sigma) : ({},{})'.format(mu_gamma_hat, std_gamma_hat))

In [ ]:


# We will fit a model for each subject
with pm.Model() as model_simple:

    # Hyperparameters for k
    # mu_kappa_hyper = pm.Beta('mu_kappa_hyper',mu=mu_kappa_hat,sigma=0.05)
    # sd_kappa_hyper = pm.Beta('sd_kappa_hyper',mu=std_kappa_hat,sigma=0.05)

    # use above mean and stdev to define kappa and gamma, the posterior from the pooled is now our prior
    kappa = pm.Beta('kappa',mu=mu_kappa_hat,sigma=std_kappa_hat,shape=np.size(np.unique(subj_id)))
    gamma = pm.HalfNormal('gamma',sigma=std_gamma_hat,shape=np.size(np.unique(subj_id)))
    
    prob = pm.Deterministic('prob', 1 / (1 + pm.math.exp(-gamma[subj_id] * ( delay_amt/(1+(kappa[subj_id]*delay_wait)) 
                                                                            - immed_amt/(1+(kappa[subj_id]*immed_wait)) ))))

    y_1 = pm.Bernoulli('y_1',p=prob,observed=choices)

    trace_prior = pm.sample(10000, tune=10000, cores=4,target_accept=0.95)

# Call the trace whatever you like. This just saves it. You don't want to run a whole model and then accidentally x-out your window or refresh or something and lose it all!
az.plot_trace(trace_prior, var_names=["kappa","gamma"],compact=False)
# This is how you get a nice array. Note that this returns a pandas DataFrame, not a numpy array. Indexing is totally different.
summary= az.summary(trace_prior,round_to=10)
fn = os.path.join(save_dir,"simple_BH_model.csv")
# Again, call it what you want (yeah - call it what you want tooooo)
print('Saving to : {}'.format(fn))
summary.to_csv(fn)